In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.chdir(globals()['_dh'][0])
os.chdir('..')
print(os.path.abspath(os.curdir))

In [ ]:
from tqdm import tqdm
import numpy as np
import random
import matplotlib.pyplot as plt

from pytorch_lightning import Trainer, seed_everything

In [3]:
nfft = 64
sig_len = 4096 + 160
osfactor = 1
cos_waves = np.exp(1j*2*np.pi*osfactor*np.arange(nfft).reshape(-1,1)/nfft*(np.arange(sig_len).reshape(1,-1)))
n_sc = 28

In [4]:
import torch
from torch import optim
from asteroid.engine import System
from torch.utils.data import TensorDataset, DataLoader

n_test = 1000

nfft = 64
sig_len = 4096 + 160
osfactor = 1
cos_waves = np.exp(1j*2*np.pi*osfactor*np.arange(nfft).reshape(-1,1)/nfft*(np.arange(sig_len).reshape(1,-1)))

n_sc = 28
def generate_sig(coeff=1, sig_type=0):
    if sig_type == 0:
        syms = coeff*(2*np.random.randint(2, size=(nfft, 1)) - 1)
    else:
        syms = coeff*(2*np.random.randint(4, size=(nfft,1)) - 3)/np.sqrt(5)
    syms[0,:] = 0
    syms[n_sc+1:,:] = 0
    syms[nfft//2+1:,:] = np.flipud(syms[1:nfft//2,:])
    sig_comp = syms * 1/np.sqrt(2*n_sc) * cos_waves
    sig = sig_comp.sum(axis=0)
    return sig, sig_comp, syms

def add_noise(sig, noise_pow=0.01):
    noise = np.sqrt(noise_pow)*np.random.randn(len(sig))
    return sig + noise

def add_interference(sig, sig_type=1):
    # interference, _, _ = generate_sig(coeff=2)
    interference, _, _ = generate_sig(coeff=4, sig_type=1)
    return sig+interference

def reconstruct_sig(syms):
    sig_comp = syms * np.sqrt(2)/np.sqrt(n_sc) * cos_waves
    sig = sig_comp.sum(axis=0)
    return sig, sig_comp

seed_everything(420123, workers=True)
np.random.seed(420123)
random.seed(420123)
all_sig, all_sig_noisy = [], [] 
for _ in tqdm(range(n_test)):
    sig, sig_comp, syms = generate_sig()
    idx = 0#np.random.randint(64)
    sig_noisy = add_interference(sig, sig_type=1)
    all_sig.append(sig[idx:idx+sig_len-160])
    all_sig_noisy.append(sig_noisy[idx:idx+sig_len-160])
    
window_len = sig_len - 160

all_sig = np.array(all_sig).reshape(-1, 1, window_len).real
all_sig_noisy = np.array(all_sig_noisy).reshape(-1, 1, window_len).real

tensor_x = torch.Tensor(all_sig_noisy)
tensor_y = torch.Tensor(all_sig)


test_dataset = TensorDataset(tensor_x,tensor_y)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers=40)

Global seed set to 420123
100%|██████████| 1000/1000 [00:01<00:00, 629.12it/s]


In [5]:
import glob
from waveunet import Waveunet
from asteroid.models import SuDORMRFImprovedNet, DPTNet, DPRNNTasNet, ConvTasNet

all_residual = {}
for idx in range(7):
    if idx == 0:
        model = Waveunet(n_src=1, n_first_filter=20, depth=5)
        model_name = 'waveunet_longksz_20filters_5depth'
    elif idx == 1:
        model = Waveunet(n_src=1, long_kernel_size=15, n_first_filter=1)
        model_name = 'waveunet0'
    elif idx == 2:
        model = SuDORMRFImprovedNet(n_src=1)
        model_name = 'sudormrf'
    elif idx ==3:
        model = DPTNet(n_src=1)
        model_name = 'dptnet'
    elif idx==4:
        model = DPRNNTasNet(n_src=1)
        model_name = 'dprnntasnet'
    elif idx==5:
        model = ConvTasNet(n_src=1)
        model_name = 'convtasnet'

    elif idx == 6:
        model = Waveunet(n_src=1, n_first_filter=20, depth=12)
        model_name = 'waveunet_longksz_20filters_12depth'
        
    folder_name = f"models/case3/{model_name}/sinsep_4096/"
    # file_list = os.listdir(folder_name)
    file_list = glob.glob(folder_name+"*")
    file_list = sorted(file_list, key=lambda t: -os.stat(t).st_mtime)
    file_list = [ fname for fname in file_list if fname.endswith('.ckpt')]
    filename = file_list[0]
    path_name = filename
    
    loss = torch.nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, amsgrad=True)
    system = System(model, optimizer, loss, test_loader, test_loader)

    ckpt = torch.load(path_name, map_location=torch.device('cpu'))
    system.load_state_dict(ckpt['state_dict'], strict=False)

    system.eval()
    with torch.no_grad():
        sig_est = system(tensor_x)
    residual = (sig_est - tensor_y).cpu().detach().numpy().squeeze()
    all_residual[model_name] = residual
    print("Model ", model_name,": MSE ", f"{10*np.log10(np.mean(np.mean(np.abs(residual)**2, axis=1))):.03f}", " ", filename)

Model  waveunet_longksz_20filters_5depth : MSE  -47.377   models/case3/waveunet_longksz_20filters_5depth/sinsep_4096/epoch=1999-step=11250000.ckpt
Model  waveunet0 : MSE  -4.663   models/case3/waveunet0/sinsep_4096/epoch=186-step=1051875.ckpt
Model  sudormrf : MSE  -12.855   models/case3/sudormrf/sinsep_4096/epoch=381-step=2148750.ckpt
Model  dptnet : MSE  -3.548   models/case3/dptnet/sinsep_4096/epoch=68-step=388125.ckpt
Model  dprnntasnet : MSE  -0.671   models/case3/dprnntasnet/sinsep_4096/epoch=21-step=123750.ckpt
Model  convtasnet : MSE  -1.009   models/case3/convtasnet/sinsep_4096/epoch=16-step=95625.ckpt
Model  waveunet_longksz_20filters_12depth : MSE  -38.625   models/case3/waveunet_longksz_20filters_12depth/sinsep_4096/epoch=816-step=4595625.ckpt


In [6]:
import pickle
pickle.dump(all_residual, open('tmp_output/case3_residual_outputs.pkl', 'wb'))